### Pair Programming - Machine Learning / Asunciones

El objetivo de este pairprgramming es que evaluéis si vuestro set de datos cumple todas las asunciones que se deben cumplir para hacer una regresión lineal. Recordamos que estas asunciones son:
- Normalidad (ya la deberíais haber evaluado).
- Homogeneidad de varianzas.
- Independencia de las variables.

Cada asunción la deberéis testar de forma visual y analítica.

In [1]:
import numpy as np
import pandas as pd
import random 

import matplotlib.pyplot as plt
import seaborn as sns


import researchpy as rp
from scipy import stats
from scipy.stats import levene
from scipy.stats import kstest

plt.rcParams["figure.figsize"] = (10,8) 

In [2]:
df = pd.read_csv("data/adult.data_limpio.csv", index_col = 0)

- Asunción de normalidad:

In [3]:
df.shape

(32560, 15)

In [4]:
df.head()

,work_class,final_weight,education,education_yrs,marital_status,occupation,relationship,ethnicity,gender,capital_gain,capital_lost,hours_week,country,salary,census
39,,,,,,,,,,,,,,,
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,32755,Bajo
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,45156,Bajo
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,39938,Bajo
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,26464,Bajo
37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,36976,Bajo


Observamos que nuestro DataFrame tiene más de 5 mil líneas, por lo tanto vamos a usar el test de Kolmogórov-Smirnov.

In [5]:
kstest(df["salary"], 'norm')

KstestResult(statistic=1.0, pvalue=0.0)

Un p-valor menor a 0.05 nos indica que debemos rechazar la hipótesis nula y que nuestros datos no son normales.

Por lo tanto, para poder introducirlo en un modelo de machine learning, deberíamos realizar una serie de cambios que veremos próximamente.

- Homogeneidad de varianzas.

Vamos a utilizar el test de Levene porque es más robusto, y se recomienda su uso para datos que no son normales.

In [6]:
df.columns

Index(['work_class', 'final_weight', 'education', 'education_yrs',
       'marital_status', 'occupation', 'relationship', 'ethnicity', 'gender',
       'capital_gain', 'capital_lost', 'hours_week', 'country', 'salary',
       'census'],
      dtype='object')

In [7]:
resultados = {}

numericas_col = df.select_dtypes(include = np.number).drop("salary", axis = 1).columns

for col in numericas_col:

    statistic, p_val = levene(df[col], df.salary, center='median')
    
    resultados[col] = p_val


In [8]:
resultados

{'final_weight': 0.0,
 'education_yrs': 0.0,
 'capital_gain': 0.0,
 'capital_lost': 0.0,
 'hours_week': 0.0}

Como podemos observar, todos los valores son menores de 0.05, por lo tanto no se cumple la asunción. Esto quiere decir que las variables son independientes unas de otras.

Quizá nos dé estos resultados porque deberíamos estar trabajando sobre los residuos, cosa que aún no sabemos lo que es.

- Independencia de las variables.

In [9]:
crosstab, test_results, expected = rp.crosstab(df["salary"], df["hours_week"],
                                               test= "chi-square",
                                               expected_freqs= True,
                                               prop= "cell")

In [10]:
crosstab.head()

hours_week                                               ...       \
hours_week          1    2    3    4    5    6    7    8    9   10  ...   90   
salary                                                              ...        
18002             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
18009             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
18010             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
18011             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
18012             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

                                                          
hours_week   91   92   94   95   96   97   98   99   All  
salary                                                    
18002       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.01  
18009       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.01  
18010       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  
18011       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.01  
18012       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  

[5 rows x 95 columns]

# warning revisar esta celda de markdown #
A la vista de los resultados anteriores, no nos extraña que continúe dándonos 0. Estas dos variables quizás no están correlacionadas porque los datos que corresponden a salary nos los hemos inventado con un random. 

In [11]:
crosstab, test_results, expected = rp.crosstab(df["salary"], df["hours_week"],
                                               test= "chi-square",
                                               expected_freqs= True,
                                               prop= "cell")

In [12]:
crosstab.head()

hours_week                                               ...       \
hours_week          1    2    3    4    5    6    7    8    9   10  ...   90   
salary                                                              ...        
18002             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
18009             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
18010             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
18011             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
18012             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

                                                          
hours_week   91   92   94   95   96   97   98   99   All  
salary                                                    
18002       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.01  
18009       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.01  
18010       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  
18011       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.01  
18012       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  

[5 rows x 95 columns]

Estos resultados nos indican que son independientes, lo que se puede deber a que hemos randomizado el salario, dado que, por norma general, sí que te pagan por horas, por lo tanto, al menos estas dos columnas, deberían estar estrechamente interrelacionadas.